In [8]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [10]:
# Load the u.data dataset
u_data = pd.read_csv('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
print(u_data.head())

   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596


In [ ]:
# Step 1: Remove unnecessary columns
u_data.drop(columns=['timestamp'], inplace=True)

# Step 2: Factorize `user_id` and `item_id` to sequential indices
u_data['user_id'], user_mapping = pd.factorize(u_data['user_id'])
u_data['item_id'], item_mapping = pd.factorize(u_data['item_id'])

# Step 3: Convert ratings into a user-item matrix (for PCT training)
user_item_matrix = u_data.pivot(index='user_id', columns='item_id', values='rating')

THRESHOLD = 3  # Ratings above this are "Lovers":1 , below are "Haters":-1, and "unknown" are 0
user_item_matrix = user_item_matrix.applymap(lambda x: 1 if x > THRESHOLD else (-1 if x > 0 else 0))

user_item_matrix



/var/folders/cv/bbbq9ykj0jg6pt6c7tx5_vjh0000gn/T/ipykernel_74784/1237650388.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  user_item_matrix = user_item_matrix.applymap(lambda x: 1 if x > THRESHOLD else (-1 if x > 0 else 0))


item_id,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
user_id,,,,,,,,,,,,,,,,,,,,,
0,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,-1,0,0,0,-1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,-1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
939,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
940,0,0,0,0,0,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Step 5: Initialize rI (Item Index) and rU (User Index)
rI = defaultdict(list)  # {item_id: [(user_id, rating)]}
rU = defaultdict(list)  # {user_id: [(item_id, rating)]}

# Populate rI and rU
for user, item, rating in u_data.itertuples(index=False):
    rI[item].append((user, rating))  # Store user-rating pairs for each item
    rU[user].append((item, rating))  # Store item-rating pairs for each user

In [13]:
# Print structures for verification
print("\n✅ rI (Item Index): Sample")
for item_id, ratings in list(rI.items())[:5]:  # Print first 5 items
    print(f"Item {item_id}: {ratings}")

print("\n✅ rU (User Index): Sample")
for user_id, ratings in list(rU.items())[:5]:  # Print first 5 users
    print(f"User {user_id}: {ratings}")


✅ rI (Item Index): Sample
Item 0: [(0, 3), (27, 3), (218, 5), (156, 3), (219, 5), (154, 4), (277, 5), (83, 4), (41, 4), (290, 4), (274, 2), (348, 5), (259, 5), (146, 2), (118, 5), (220, 5), (352, 4), (19, 4), (137, 4), (334, 4), (317, 4), (175, 3), (189, 4), (444, 5), (354, 4), (280, 5), (76, 5), (286, 5), (465, 2), (169, 4), (413, 3), (229, 5), (491, 1), (511, 5), (304, 5), (260, 3), (201, 5), (445, 1), (523, 4), (540, 5), (534, 3), (409, 4), (558, 5), (594, 4), (35, 1), (631, 4), (150, 5), (520, 5), (607, 3), (320, 4), (620, 4), (293, 4), (647, 5), (482, 5), (121, 4), (458, 4), (687, 5), (199, 4), (305, 4), (737, 4), (184, 5), (788, 5), (459, 3), (763, 4), (735, 4), (578, 4), (667, 4), (45, 5), (9, 4), (230, 4), (584, 5), (438, 5), (665, 5), (766, 5), (321, 3), (624, 5), (472, 3), (699, 5), (262, 3), (859, 4), (80, 4), (593, 4), (872, 3), (798, 5), (168, 5), (757, 4), (738, 5), (877, 2), (650, 4), (114, 5), (902, 4), (935, 3), (760, 4), (530, 4), (691, 4), (529, 5), (571, 5), (752, 

In [8]:
import os
import pickle

# Ensure the "data/" directory exists before saving files
output_dir = "/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/ml-100k/data"
os.makedirs(output_dir, exist_ok=True)  # Creates the directory if it does not exist

# Save user-item matrix
user_item_matrix.to_csv(os.path.join(output_dir, "user_item_matrix.csv"), index=True)

# Save the mappings to retrieve original user/item IDs later
user_mapping_dict = dict(enumerate(user_mapping))
item_mapping_dict = dict(enumerate(item_mapping))

with open(os.path.join(output_dir, "user_mapping.pkl"), "wb") as f:
    pickle.dump(user_mapping_dict, f)
with open(os.path.join(output_dir, "item_mapping.pkl"), "wb") as f:
    pickle.dump(item_mapping_dict, f)
